In [10]:
import os
import re
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# ——— GPU & MEMORY CONFIG —————————————————————————————————
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
tf.keras.mixed_precision.set_global_policy('mixed_float16')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using GPU: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")
else:
    print("No GPU found. Using CPU instead.")

def clear_memory():
    K.clear_session()
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

# ——— USER SETTINGS —————————————————————————————————————
selected_optimizer = 'adamw'
train_dir      = './amla_images/train'
validation_dir = './amla_images/val'
test_dir       = './amla_images/test'
checkpoint_dir = "./checkpoints_amla_opt2"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint_amla_opt2_epoch_{epoch:02d}.keras")

# ——— DATA ————————————————————————————————————————————
train_datagen    = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 4  # ↓ Smaller batch to avoid OOM

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=batch_size, class_mode='binary'
)
val_gen = val_test_datagen.flow_from_directory(
    validation_dir, target_size=(224, 224), batch_size=batch_size, class_mode='binary'
)
test_gen = val_test_datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=batch_size, class_mode='binary', shuffle=False
)

# ——— MODEL ————————————————————————————————————————————
inp      = Input(shape=(224, 224, 3))
vgg_base = VGG19(weights='imagenet', include_top=False, input_tensor=inp)
dn_base  = DenseNet121(weights='imagenet', include_top=False, input_tensor=inp)

for l in vgg_base.layers: l.trainable = False
for l in dn_base.layers:  l.trainable = False

x1 = GlobalAveragePooling2D()(vgg_base.output)
x2 = GlobalAveragePooling2D()(dn_base.output)
x  = Concatenate()([x1, x2])
x  = Dense(1024, activation='relu')(x)
out= Dense(1, activation='sigmoid', dtype='float32')(x)  # For mixed precision

model = Model(inputs=inp, outputs=out)

# ——— CHECKPOINT UTIL ————————————————————————————————————
def get_latest_checkpoint():
    files = [f for f in os.listdir(checkpoint_dir) if f.startswith("model_checkpoint_amla_opt2_epoch_")]
    if not files: return None, 0
    epochs = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in files]
    e_max  = max(epochs)
    return os.path.join(checkpoint_dir, f"model_checkpoint_amla_opt2_epoch_{e_max:02d}.keras"), e_max

latest_ckpt, last_epoch = get_latest_checkpoint()
if latest_ckpt and os.path.exists(latest_ckpt):
    print("Loading checkpoint:", latest_ckpt)
    model = load_model(latest_ckpt)
else:
    print("No checkpoint found, starting fresh")

# ——— OPTIMIZERS ————————————————————————————————————————
def get_optimizer(name, lr):
    if name == 'adamw':
        return tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=1e-5)
    elif name == 'nadam':
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    else:
        return tf.keras.optimizers.Adam(learning_rate=lr)

# ——— CALLBACK ————————————————————————————————————————
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=False,
    verbose=1
)

# ——— TRAINING PHASES ————————————————————————————————————
ph1, ph2, ph3 = 10, 5, 5

if last_epoch < ph1:
    model.compile(
        optimizer=get_optimizer(selected_optimizer, 1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // batch_size,
        validation_data=val_gen,
        validation_steps=val_gen.samples // batch_size,
        initial_epoch=last_epoch,
        epochs=ph1,
        callbacks=[checkpoint_cb]
    )
    last_epoch = ph1
    clear_memory()

if last_epoch < ph1 + ph2:
    for l in vgg_base.layers[-5:]: l.trainable = True
    for l in dn_base.layers[-5:]:  l.trainable = True
    model.compile(
        optimizer=get_optimizer(selected_optimizer, 1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // batch_size,
        validation_data=val_gen,
        validation_steps=val_gen.samples // batch_size,
        initial_epoch=last_epoch,
        epochs=ph1 + ph2,
        callbacks=[checkpoint_cb]
    )
    last_epoch = ph1 + ph2
    clear_memory()

if last_epoch < ph1 + ph2 + ph3:
    for l in vgg_base.layers: l.trainable = True
    for l in dn_base.layers:  l.trainable = True
    model.compile(
        optimizer=get_optimizer(selected_optimizer, 1e-6),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // batch_size,
        validation_data=val_gen,
        validation_steps=val_gen.samples // batch_size,
        initial_epoch=last_epoch,
        epochs=ph1 + ph2 + ph3,
        callbacks=[checkpoint_cb]
    )
    clear_memory()

# Recompile the model after clearing session
model.compile(
    optimizer=get_optimizer(selected_optimizer, 1e-6),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ——— EVALUATE ————————————————————————————————————————
test_loss, test_acc = model.evaluate(test_gen)
val_loss, val_acc  = model.evaluate(val_gen)
print(f"Test   → loss={test_loss:.4f}, acc={test_acc:.4f}")
print(f"Val    → loss={val_loss:.4f}, acc={val_acc:.4f}")


Using GPU: ['/physical_device:GPU:0']
Found 11124 images belonging to 2 classes.
Found 2386 images belonging to 2 classes.
Found 2386 images belonging to 2 classes.
Loading checkpoint: ./checkpoints_amla_opt2/model_checkpoint_amla_opt2_epoch_20.keras


2025-04-27 17:35:05.753788: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3891', 228 bytes spill stores, 228 bytes spill loads



596/597 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 4.7060e-05

2025-04-27 17:35:49.068168: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3891', 228 bytes spill stores, 228 bytes spill loads



597/597 ━━━━━━━━━━━━━━━━━━━━ 80s 110ms/step - accuracy: 1.0000 - loss: 4.7572e-05
597/597 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - accuracy: 0.9995 - loss: 0.0023
Test   → loss=0.0002, acc=1.0000
Val    → loss=0.0032, acc=0.9992


In [7]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model_at_epoch(epoch):
    model_path = os.path.join(checkpoint_dir, f"model_checkpoint_amla_opt2_epoch_{epoch:02d}.h5")
    if not os.path.exists(model_path):
        print(f"No model found at epoch {epoch}")
        return
    
    # Load the model
    model = load_model(model_path)
    print(f"Loaded model from epoch {epoch}")

    # Evaluate basic metrics (loss and accuracy)
    test_loss, test_acc = model.evaluate(test_gen)
    val_loss, val_acc   = model.evaluate(val_gen)

    print(f"Epoch {epoch} - Test → loss={test_loss:.4f}, acc={test_acc:.4f}")
    print(f"Epoch {epoch} - Val  → loss={val_loss:.4f}, acc={val_acc:.4f}")

    # Predict on Test Set
    test_gen.reset()
    preds = model.predict(test_gen, verbose=1)
    preds = np.round(preds).astype(int).flatten()

    # True labels
    true_labels = test_gen.classes

    # Classification Report
    report = classification_report(true_labels, preds, target_names=test_gen.class_indices.keys(), digits=4)
    print(f"\nClassification Report on Test Set at Epoch {epoch}:\n")
    print(report)





In [ ]:
evaluate_model_at_epoch(20)


No model found at epoch 20
